In [16]:
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
import datetime as dt

In [24]:
# БЛОК С ИМПОРТОМ ДАННЫХ
data=pd.read_csv("./Data/casc-resto.csv",sep=';') # Считываем данные из файла
data['RKDate']=pd.to_datetime(data['RKDate'])
fix_date=(datetime.strptime('2017-07-01', '%Y-%m-%d'))
# Группируем данные по идентификатору клиента и смотрим на данные


In [29]:
# БЛОК С РАСЧЕТОМ ПОКАЗАТЕЛЕЙ
def agg(group): # функция, в которой считаются все показатели
    bought=0 # переменная для покупал/не покупал
    recency=10000
    spent = 0
    visitedTimes = 1
    nonCashSpent = 0
    visitedTimes1 = 1
    visitsBeforeFix=0
    firstVisitDate=fix_date
    for index, row in group.iterrows(): # пробегаемся по датам в каждой группе 
        if row['RKDate'] <=  datetime.strptime('2017-12-31', '%Y-%m-%d')  and row['RKDate'] >= fix_date: # сравниваем даты
            bought=1 # покупка была совершена в промежуток времени 
            
        if row['RKDate'] <= fix_date and (fix_date - row['RKDate']).days < recency: 
            recency = (fix_date - row['RKDate']).days # считаем количество дней от даты покупки/визита до фиксированной даты
                
        if row['RKDate'] <= fix_date:
            visitsBeforeFix=visitsBeforeFix+1 # считаем дату самого недавнего визита до фиксированной даты
            spent+=row['SummAfterPointsUsage'] # считаем сколько всего было потрачено
        if row['RKDate']<= firstVisitDate:
            firstVisitDate=row['RKDate'] # считаем дату первого визита 
            
#        if (row['RKDate'] <= fix_date):
#            visitedTimes1+=1
#            nonCashSpent+= ( row['SummBasic'] - row['SummAfterPointsUsage'] )
#            someVar= nonCashSpent / visitedTimes1
    if visitsBeforeFix == 0: # если не было визитов до фиксированной даты, то частота равна 0
        frequency=0
    months  = 1 + (relativedelta.relativedelta(fix_date,firstVisitDate)).months # считаем сколько месяцев прошло от первого визита до фиксированной даты
    frequency= visitsBeforeFix/ months # считаем частоту
    
    MonetaryValue= spent / visitsBeforeFix # считаем средний чек покупки за время до фиксированной даты      
    return bought,recency,frequency #,,MonetaryValue,someVar

    

def func(group): #функция, в которой все посчитанные показатели собираются в один массив
    bought, recency,frequency=agg(group)
    d = {}
    d['Bought_Or_Not'] = bought
    d['Recency'] = recency
    d['Frequency'] = frequency
#    d['Monetary_Value'] = MonetaryValue
#    d['Some_Variable'] = someVar
   
    return pd.Series(d, index=['Bought_Or_Not', 'Recency','Frequency'])
#,'Frequency','Monetary_Value','Some_Variable'
groupedData = data.groupby("CustomerID").apply(func) # группируем записи по клиенту и применяем функцию, в которой все считается
aggregatedData= groupedData.reset_index()
print(aggregatedData)



      CustomerID  Bought_Or_Not  Recency  Frequency
0        2728046            0.0    160.0   8.142857
1        2728088            0.0    720.0   4.857143
2        2728089            1.0     15.0   4.714286
3        2728095            0.0    177.0  14.142857
4        2728107            1.0    115.0   3.000000
...          ...            ...      ...        ...
9995     2913103            0.0    401.0   9.750000
9996     2913106            0.0    126.0  46.750000
9997     2913114            0.0     95.0  27.750000
9998     2913123            1.0      8.0  58.000000
9999     2913132            1.0     44.0   2.571429

[10000 rows x 4 columns]


In [28]:
# БЛОК СО СЛИЯНИЕМ ДАННЫХ
dataToMergeWith =pd.read_csv("./Data/CASC_constant.csv",sep=';') # Считываем данные из файла для слияния

MergedData=pd.merge(aggregatedData, dataToMergeWith, left_on='CustomerID', right_on='CustomerId') # Делаем слияние двух таблиц

Sex=pd.get_dummies(MergedData['Sex'], drop_first=True) # создадим новый столбец, в котором используем фиктивные переменные: мужской пол =1 ,женский пол = 0

MergedData.drop(['Sex'],axis=1,inplace=True) # Удаляем столбец с полом, потому что там прописано м и ж, а нам нужно 1 или 0

MergedData=pd.concat([MergedData,Sex]) # присоединяем новый столбец с полом в нужном формате к нашей таблице

MergedData['ActivationDate'] = pd.to_datetime(MergedData['ActivationDate']) # переведем формат из строки в дату

MergedData['ActivationDate']=MergedData['ActivationDate'].map(dt.datetime.toordinal) # сделаем дату числом

print(MergedData)

      CustomerID  Bought_Or_Not  Recency
0        2728046              0      160
1        2728088              0      720
2        2728089              1       15
3        2728095              0      177
4        2728107              1      115
...          ...            ...      ...
9995     2913103              0      401
9996     2913106              0      126
9997     2913114              0       95
9998     2913123              1        8
9999     2913132              1       44

[10000 rows x 3 columns]


In [ ]:
# БЛОК С ОБУЧЕНИЕМ МОДЕЛИ
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

X_data=MergedData.drop('Bought_or_Not',axis=1) # исключаем из данных зависимую переменную

x_train, x_test, y_train, y_test = train_test_split(X_data, MergedData['Bought_or_Not'], test_size=0.2, random_state=42) # разделим данные, тестовая выборка составит 20%
logreg=LogisticRegression() # создадим логистическую регрессию
logreg.fit(x_train, y_train) # применим функцию fit на обучающей выборке
y_pred = logreg.predict(x_test) # применим функцию predict для предсказания меток
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))